## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import ipywidgets as widgets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,77.00,83,75,11.50,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.47,74,100,13.20,overcast clouds
2,2,Kaitangata,NZ,-46.2817,169.8464,50.25,93,2,3.51,clear sky
3,3,Busselton,AU,-33.6500,115.3333,60.01,93,100,1.01,light rain
4,4,Fort Nelson,CA,58.8053,-122.7002,-36.40,67,20,2.30,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature? '))
max_temp = float(input('What is the maximum temperature? '))

What is the minimum temperature? 75
What is the maximum temperature? 87


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,77.00,83,75,11.50,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.47,74,100,13.20,overcast clouds
10,10,Sinnamary,GF,5.3833,-52.9500,80.17,84,92,14.94,overcast clouds
11,11,Hithadhoo,MV,-0.6000,73.0833,81.99,76,100,14.25,overcast clouds
15,15,Vao,NC,-22.6667,167.4833,77.31,87,1,8.12,clear sky
...,...,...,...,...,...,...,...,...,...,...
570,570,Qandala,SO,11.4720,49.8728,75.51,74,15,2.80,few clouds
571,571,Sao Gabriel Da Cachoeira,BR,-0.1303,-67.0892,82.40,78,75,3.44,broken clouds
573,573,Outram,MY,1.2829,103.8379,80.60,78,75,4.61,broken clouds
579,579,High Rock,BS,26.6208,-78.2833,77.00,83,75,10.36,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,77.00,83,75,11.50,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.47,74,100,13.20,overcast clouds
10,10,Sinnamary,GF,5.3833,-52.9500,80.17,84,92,14.94,overcast clouds
11,11,Hithadhoo,MV,-0.6000,73.0833,81.99,76,100,14.25,overcast clouds
15,15,Vao,NC,-22.6667,167.4833,77.31,87,1,8.12,clear sky
...,...,...,...,...,...,...,...,...,...,...
570,570,Qandala,SO,11.4720,49.8728,75.51,74,15,2.80,few clouds
571,571,Sao Gabriel Da Cachoeira,BR,-0.1303,-67.0892,82.40,78,75,3.44,broken clouds
573,573,Outram,MY,1.2829,103.8379,80.60,78,75,4.61,broken clouds
579,579,High Rock,BS,26.6208,-78.2833,77.00,83,75,10.36,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,77.00,broken clouds,-33.0153,27.9116,
1,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,
10,Sinnamary,GF,80.17,overcast clouds,5.3833,-52.9500,
11,Hithadhoo,MV,81.99,overcast clouds,-0.6000,73.0833,
15,Vao,NC,77.31,clear sky,-22.6667,167.4833,
17,Karratha,AU,78.82,clear sky,-20.7377,116.8463,
21,Souillac,MU,80.60,few clouds,-20.5167,57.5167,
23,Padang,ID,77.00,few clouds,-0.9492,100.3543,
26,Merauke,ID,80.24,overcast clouds,-8.4667,140.3333,
29,Pangkalanbuun,ID,75.40,light rain,-2.6833,111.6167,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
        

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,77.00,broken clouds,-33.0153,27.9116,Tu Casa
10,Sinnamary,GF,80.17,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
15,Vao,NC,77.31,clear sky,-22.6667,167.4833,Hotel Kou-Bugny
17,Karratha,AU,78.82,clear sky,-20.7377,116.8463,ibis Styles Karratha
21,Souillac,MU,80.60,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
...,...,...,...,...,...,...,...
561,Denpasar,ID,77.00,few clouds,-8.6500,115.2167,ASTON Denpasar Hotel & Convention Center
569,Marawi,PH,75.99,overcast clouds,7.9986,124.2928,Derogongan Residence
571,Sao Gabriel Da Cachoeira,BR,82.40,broken clouds,-0.1303,-67.0892,Alarne - Área De Lazer Do Alto Rio Negro
573,Outram,MY,80.60,broken clouds,1.2829,103.8379,Grand Copthorne Waterfront


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# Add a heatmap of temperature for the vacation spots.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))